In [5]:
import pandas as pd

# Load Titanic dataset
df = pd.read_csv("titanic.csv")

# Select relevant features
features = ["Pclass", "Sex", "Age", "SibSp", "Parch", "Fare"]
label = "Survived"

# Fill missing values
df["Age"].fillna(df["Age"].median(), inplace=True)
df["Fare"].fillna(df["Fare"].median(), inplace=True)

# Convert categorical variables
df["Sex"] = df["Sex"].map({"male": 0, "female": 1})

# Normalize numerical columns (Min-Max Scaling)
for col in ["Age", "Fare"]:
    df[col] = (df[col] - df[col].min()) / (df[col].max() - df[col].min())

# Extract features and labels
X = df[features].values.tolist()  # Convert to pure Python list
y = df[label].values.tolist()      # Convert labels to list

In [29]:
import random
import math

# Sigmoid activation function
def sigmoid(x):
    """Numerically stable sigmoid function to prevent overflow issues."""
    if x < -500:
        return 0  # Avoid overflow for large negative values
    elif x > 500:
        return 1  # Avoid underflow for large positive values
    return 1 / (1 + math.exp(-x))

def sigmoid_derivative(x):
    return x * (1 - x)  # Derivative of sigmoid

# Initialize network with random weights
def initialize_network(input_size, hidden_size, output_size):
    network = {
        # ✅ Fix: Ensure each row in W1 corresponds to an input feature
        "W1": [[random.uniform(-1, 1) for _ in range(hidden_size)] for _ in range(input_size)],
        "B1": [random.uniform(-1, 1) for _ in range(hidden_size)],

        # ✅ Fix: Ensure each row in W2 corresponds to a hidden neuron
        "W2": [[random.uniform(-1, 1) for _ in range(output_size)] for _ in range(hidden_size)],
        "B2": [random.uniform(-1, 1) for _ in range(output_size)],
    }
    return network

# Forward pass
def forward_pass(network, inputs):
    # ✅ Fix: Ensure correct iteration over the hidden layer
    hidden_layer_input = [
        sum(inputs[j] * network["W1"][j][i] for j in range(len(inputs))) + network["B1"][i]
        for i in range(len(network["B1"]))
    ]
    hidden_layer_output = [sigmoid(x) for x in hidden_layer_input]

    # ✅ Fix: Ensure correct iteration over the output layer
    output_layer_input = [
        sum(hidden_layer_output[h] * network["W2"][h][o] for h in range(len(hidden_layer_output))) + network["B2"][o]
        for o in range(len(network["B2"]))
    ]
    output_layer_output = [sigmoid(x) for x in output_layer_input]

    return hidden_layer_output, output_layer_output

# Backpropagation
def backward_pass(network, inputs, hidden_output, output_output, expected, learning_rate=0.1):
    output_errors = [(expected[i] - output_output[i]) for i in range(len(expected))]
    output_deltas = [output_errors[i] * sigmoid_derivative(output_output[i]) for i in range(len(expected))]

    hidden_errors = [sum(output_deltas[o] * network["W2"][h][o] for o in range(len(output_deltas))) for h in range(len(hidden_output))]
    hidden_deltas = [hidden_errors[h] * sigmoid_derivative(hidden_output[h]) for h in range(len(hidden_output))]

    # Update output layer weights
    for h in range(len(network["W2"])):
        for o in range(len(network["W2"][h])):
            network["W2"][h][o] += learning_rate * output_deltas[o] * hidden_output[h]

    # Update output layer biases
    for o in range(len(network["B2"])):
        network["B2"][o] += learning_rate * output_deltas[o]

    # Update hidden layer weights
    for i in range(len(network["W1"])):
        for h in range(len(network["W1"][i])):
            network["W1"][i][h] += learning_rate * hidden_deltas[h] * inputs[i]

    # Update hidden layer biases
    for h in range(len(network["B1"])):
        network["B1"][h] += learning_rate * hidden_deltas[h]

# Train the network
def train_network(network, X, y, epochs=1000, learning_rate=0.1):
    for epoch in range(epochs):
        total_loss = 0
        for inputs, expected in zip(X, y):
            expected = [expected]  # Convert to list for compatibility
            hidden_output, output_output = forward_pass(network, inputs)
            backward_pass(network, inputs, hidden_output, output_output, expected, learning_rate)
            total_loss += sum((expected[i] - output_output[i]) ** 2 for i in range(len(expected)))  # MSE loss

        if epoch % 100 == 0:
            print(f"Epoch {epoch}: Loss = {total_loss / len(X)}")

# Initialize and train
network = initialize_network(input_size=len(X[0]), hidden_size=15, output_size=1)
train_network(network, X, y, epochs=1000, learning_rate=0.01)

Epoch 0: Loss = 0.26123853039547285
Epoch 100: Loss = 0.14270303087257555
Epoch 200: Loss = 0.13943654200885547
Epoch 300: Loss = 0.1367286314636621
Epoch 400: Loss = 0.13476712315046682
Epoch 500: Loss = 0.13345825266375086
Epoch 600: Loss = 0.13250017533794897
Epoch 700: Loss = 0.13171858983753543
Epoch 800: Loss = 0.13103983000798963
Epoch 900: Loss = 0.130431535672393


In [19]:
# Evaluate on test data
correct = 0
for inputs, expected in zip(X, y):
    _, output_output = forward_pass(network, inputs)
    prediction = 1 if output_output[0] > 0.5 else 0
    correct += int(prediction == expected)

print(f"✅ Accuracy: {correct / len(X) * 100:.2f}%")

✅ Accuracy: 83.16%
